<a href="https://colab.research.google.com/github/QiaoYe-Learning/NLP-classwork/blob/main/Assignment_1_Simple_Sentiment_Analysis_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Simple Sentiment Analysis

## Preparing Data

In [1]:
# download dataset
!pip install torchtext==0.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00


In [13]:
# initial setting
import torch
from torchtext import data

SEED = 1234 # random seed

torch.manual_seed(SEED) # normalize random seed
torch.backends.cudnn.deterministic = True # enture CUDA's behavior is stable（确定性）

TEXT = data.Field(tokenize = 'spacy', # split word using spacy
                  tokenizer_language = 'en_core_web_sm', # enture spacy using English_model
                  include_lengths = True, # include the length of sentence when return the data
                  pad_first=True) # add a mark before the sequence
LABEL = data.LabelField(dtype = torch.float) # specfies the tyoe of label data is 'float'(binary classfication and regression)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Another handy feature of TorchText is that it has support for common datasets used in natural language processing (NLP).

The following code automatically downloads the IMDb dataset and splits it into the canonical train/test splits as `torchtext.datasets` objects. It process the data using the `Fields` we have previously defined. The IMDb dataset consists of 50,000 movie reviews, each marked as being a positive or negative review.

In [20]:
from torchtext import datasets # pre-define dataset supported by torchtext

training_data, test_data = datasets.IMDB.splits(TEXT, LABEL) # load dataset-IMDB(binary classfication task)

We can see how many examples are in each split by checking their length.

In [21]:
print(f'Number of training examples: {len(training_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


We can also check an example.

In [22]:
print(vars(training_data.examples[0])) # show the 1st example in the train

{'text': ['Scott', 'Henderson', '(', 'Alan', 'Curtis', ')', 'is', 'unjustly', 'accused', 'of', 'killing', 'his', 'unfaithful', 'wife', '.', 'The', 'night', 'she', 'was', 'murdered', 'he', 'was', 'out', 'with', 'a', 'mystery', 'woman', 'who', 'refused', 'to', 'give', 'him', 'her', 'name', '.', 'After', 'accused', 'of', 'the', 'murder', ',', 'the', 'police', 'visit', 'all', 'the', 'places', 'he', 'had', 'been', 'with', 'her', ',', 'but', 'people', 'only', 'remember', 'him', 'being', 'alone', '.', 'He', "'s", 'sentenced', 'to', 'die', 'and', 'his', 'secretary', '(', 'Ella', 'Raines', ')', 'sets', 'out', 'to', 'find', 'the', 'murderer', 'herself', 'because', 'she', 'loves', 'him.<br', '/><br', '/>This', 'was', 'made', 'as', 'a', 'B', 'film', 'from', 'Universal', '(', 'look', 'at', 'the', 'cast', '--', 'all', 'character', 'actors', 'and', 'one', 'star', '--', 'Franchot', 'Tone', '--', 'on', 'the', 'decline', ')', '.', 'The', 'budget', 'was', 'small', 'and', 'the', 'cast', 'mostly', 'unknown

The IMDb dataset only has train/test splits, so we need to create a validation set. We can do this with the `.split()` method.

By default this splits 70/30, however by passing a `split_ratio` argument, we can change the ratio of the split, i.e. a `split_ratio` of 0.8 would mean 80% of the examples make up the training set and 20% make up the validation set.

We also pass our random seed to the `random_state` argument, ensuring that we get the same train/validation split each time.

In [23]:
import random

train_data, valid_data = training_data.split(random_state = random.seed(SEED)) # Default 70/30
# train_data, valid_data = train_data.split(split_ratio=0.8, random_state=random.seed(SEED)) 80/20 = train/valid

Again, we'll view how many examples are in each split.

In [25]:
print(f'Number of training examples: {len(training_data)}')
print(f'Number of train examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of train examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


Next, we have to build a _vocabulary_. This is a effectively a look up table where every unique word in your data set has a corresponding _index_ (an integer).

We do this as our machine learning model cannot operate on strings, only numbers. Each _index_ is used to construct a _one-hot_ vector for each word. A one-hot vector is a vector where all of the elements are 0, except one, which is 1, and dimensionality is the total number of unique words in your vocabulary, commonly denoted by $V$.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment5.png?raw=1)

The number of unique words in our training set is over 100,000, which means that our one-hot vectors will have over 100,000 dimensions! This will make training slow and possibly won't fit onto your GPU (if you're using one).

There are two ways effectively cut down our vocabulary, we can either only take the top $n$ most common words or ignore words that appear less than $m$ times. We'll do the former, only keeping the top 25,000 words.

What do we do with words that appear in examples but we have cut from the vocabulary? We replace them with a special _unknown_ or `<unk>` token. For example, if the sentence was "This film is great and I love it" but the word "love" was not in the vocabulary, it would become "This film is great and I `<unk>` it".

The following builds the vocabulary, only keeping the most common `max_size` tokens.

In [26]:
# build a vocabulary
MAX_VOCAB_SIZE = 25_000 # max size of vocabulary, only top 25000, other is <unk>

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE) # build train vocabulary
LABEL.build_vocab(train_data) # build vicabulary for the label fields <unk>,<pad>

Why do we only build the vocabulary on the training set? When testing any machine learning system you do not want to look at the test set in any way. We do not include the validation set as we want it to reflect the test set as much as possible.

In [27]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(f"Index of <unk>: {TEXT.vocab.stoi['<unk>']}")
print(f"Index of <pad>: {TEXT.vocab.stoi['<pad>']}") # using <pad> to fill the sentence in order to ensure the same size

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
Index of <unk>: 0
Index of <pad>: 1


Why is the vocab size 25002 and not 25000? One of the addition tokens is the `<unk>` token and the other is a `<pad>` token.

When we feed sentences into our model, we feed a _batch_ of them at a time, i.e. more than one at a time, and all sentences in the batch need to be the same size. Thus, to ensure each sentence in the batch is the same size, any shorter than the longest within the batch are padded.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment6.png?raw=1)

We can also view the most common words in the vocabulary and their frequencies.

In [35]:
print(TEXT.vocab.freqs.most_common(20)) # top 20 common words

[('the', 203876), (',', 192811), ('.', 166221), ('and', 110231), ('a', 109704), ('of', 101138), ('to', 94235), ('is', 76501), ('in', 61848), ('I', 54382), ('it', 53259), ('that', 49340), ('"', 44686), ("'s", 43449), ('this', 42379), ('-', 37601), ('/><br', 35747), ('was', 34946), ('as', 30640), ('with', 29883)]


We can also see the vocabulary directly using either the `stoi` (**s**tring **to** **i**nt) or `itos` (**i**nt **to**  **s**tring) method.

In [34]:
print(TEXT.vocab.itos[:12]) # The first 12 'words' in the vocabulary

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', 'I']


We can also check the labels, ensuring 0 is for negative and 1 is for positive.

In [38]:
print(LABEL.vocab.stoi) # <unk> - <neg> ; <pad> - <pos>

defaultdict(None, {'neg': 0, 'pos': 1})


In [39]:
TEXT.__dict__  # View all properties of the TEXT object and their corresponding values

{'sequential': True,
 'use_vocab': True,
 'init_token': None,
 'eos_token': None,
 'unk_token': '<unk>',
 'fix_length': None,
 'dtype': torch.int64,
 'preprocessing': None,
 'postprocessing': None,
 'lower': False,
 'tokenizer_args': ('spacy', 'en_core_web_sm'),
 'tokenize': functools.partial(<function _spacy_tokenize at 0x7d7cac85dc60>, spacy=<spacy.lang.en.English object at 0x7d7c583b1780>),
 'include_lengths': True,
 'batch_first': False,
 'pad_token': '<pad>',
 'pad_first': True,
 'truncate_first': False,
 'stop_words': None,
 'is_target': False,
 'vocab': <torchtext.vocab.Vocab at 0x7d7c0c8e51e0>}

The final step of preparing the data is creating the iterators. We iterate over these in the training/evaluation loop, and they return a batch of examples (indexed and converted into tensors) at each iteration.

We'll use a `BucketIterator` which is a special type of iterator that will return a batch of examples where each example is of a similar length, minimizing the amount of padding per example.

We also want to place the tensors returned by the iterator on the GPU (if you're using one). PyTorch handles this using `torch.device`, we then pass this device to the iterator.

In [40]:
# using BucketIterator to create data iterator for train, valid, test to processes data in batch
BATCH_SIZE = 64 # size of samples per batch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # device choice, preferential using GPU(cuda)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True, # sentences in each batch are sorted by their length, useful to reduce the using of <pad>
    device = device)

## Build the Model

The next stage is building the model that we'll eventually train and evaluate.

There is a small amount of boilerplate code when creating models in PyTorch, note how our `RNN` class is a sub-class of `nn.Module` and the use of `super`.

Within the `__init__` we define the _layers_ of the module. Our three layers are an _embedding_ layer, our RNN, and a _linear_ layer. All layers have their parameters initialized to random values, unless explicitly specified.

The embedding layer is used to transform our sparse one-hot vector (sparse as most of the elements are 0) into a dense embedding vector (dense as the dimensionality is a lot smaller and all the elements are real numbers). This embedding layer is simply a single fully connected layer. As well as reducing the dimensionality of the input to the RNN, there is the theory that words which have similar impact on the sentiment of the review are mapped close together in this dense vector space. For more information about word embeddings, see [here](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/).

The RNN layer is our RNN which takes in our dense vector and the previous hidden state $h_{t-1}$, which it uses to calculate the next hidden state, $h_t$.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment7.png?raw=1)

Finally, the linear layer takes the final hidden state and feeds it through a fully connected layer, $f(h_T)$, transforming it to the correct output dimension.

The `forward` method is called when we feed examples into our model.

Each batch, `text`, is a tensor of size _**[sentence length, batch size]**_. That is a batch of sentences, each having each word converted into a one-hot vector.

You may notice that this tensor should have another dimension due to the one-hot vectors, however PyTorch conveniently stores a one-hot vector as it's index value, i.e. the tensor representing a sentence is just a tensor of the indexes for each token in that sentence. The act of converting a list of tokens into a list of indexes is commonly called *numericalizing*.

The input batch is then passed through the embedding layer to get `embedded`, which gives us a dense vector representation of our sentences. `embedded` is a tensor of size _**[sentence length, batch size, embedding dim]**_.

`embedded` is then fed into the RNN. In some frameworks you must feed the initial hidden state, $h_0$, into the RNN, however in PyTorch, if no initial hidden state is passed as an argument it defaults to a tensor of all zeros.

The RNN returns 2 tensors, `output` of size _**[sentence length, batch size, hidden dim]**_ and `hidden` of size _**[1, batch size, hidden dim]**_. `output` is the concatenation of the hidden state from every time step, whereas `hidden` is simply the final hidden state. We verify this using the `assert` statement. Note the `squeeze` method, which is used to remove a dimension of size 1.

Finally, we feed the last hidden state, `hidden`, through the linear layer, `fc`, to produce a prediction.

In [43]:
import torch.nn as nn # Neural Network

class RNN(nn.Module): # inherited from nn.Module
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim): # vocabulary size, embedding_dim, hidden_dim, classification number

        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.rnn = nn.RNN(embedding_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text, text_lengths):
        #text = [sentence length, batch size], text_lengths: real length

        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]

        output, hidden = self.rnn(embedded)
        #output = [sent len, batch size, hid dim] - The hidden status of each time step, containing information about all time steps.
        #hidden = [1, batch size, hid dim] - The hidden state of the last time step

        #assert torch.equal(output[-1,:,:], hidden.squeeze(0)) - Verify that the output of the last time step is equal to the hidden state

        return self.fc(hidden.squeeze(0))

We now create an instance of our RNN class.

The input dimension is the dimension of the one-hot vectors, which is equal to the vocabulary size.

The embedding dimension is the size of the dense word vectors. This is usually around 50-250 dimensions, but depends on the size of the vocabulary.

The hidden dimension is the size of the hidden states. This is usually around 100-500 dimensions, but also depends on factors such as on the vocabulary size, the size of the dense vectors and the complexity of the task.

The output dimension is usually the number of classes, however in the case of only 2 classes the output value is between 0 and 1 and thus can be 1-dimensional, i.e. a single scalar real number.

In [45]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

Let's also create a function that will tell us how many trainable parameters our model has so we can compare the number of parameters across different models.

In [47]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


## Train the Model

Now we'll set up the training and then train the model.

First, we'll create an optimizer. This is the algorithm we use to update the parameters of the module. Here, we'll use _stochastic gradient descent_ (SGD). The first argument is the parameters will be updated by the optimizer, the second is the learning rate, i.e. how much we'll change the parameters by when we do a parameter update.

In [50]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3) # SGD optimizer - Stochastic gradient descent

Next, we'll define our loss function. In PyTorch this is commonly called a criterion.

The loss function here is _binary cross entropy with logits_.

Our model currently outputs an unbound real number. As our labels are either 0 or 1, we want to restrict the predictions to a number between 0 and 1. We do this using the _sigmoid_ or _logit_ functions.

We then use this this bound scalar to calculate the loss using binary cross entropy.

The `BCEWithLogitsLoss` criterion carries out both the sigmoid and the binary cross entropy steps.

In [51]:
criterion = nn.BCEWithLogitsLoss() # Loss function of binary task

Using `.to`, we can place the model and the criterion on the GPU (if we have one).

In [52]:
model = model.to(device)
criterion = criterion.to(device)

Our criterion function calculates the loss, however we have to write our function to calculate the accuracy.

This function first feeds the predictions through a sigmoid layer, squashing the values between 0 and 1, we then round them to the nearest integer. This rounds any value greater than 0.5 to 1 (a positive sentiment) and the rest to 0 (a negative sentiment).

We then calculate how many rounded predictions equal the actual labels and average it across the batch.

In [54]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

The `train` function iterates over all examples, one batch at a time.

`model.train()` is used to put the model in "training mode", which turns on _dropout_ and _batch normalization_. Although we aren't using them in this model, it's good practice to include it.

For each batch, we first zero the gradients. Each parameter in a model has a `grad` attribute which stores the gradient calculated by the `criterion`. PyTorch does not automatically remove (or "zero") the gradients calculated from the last gradient calculation, so they must be manually zeroed.

We then feed the batch of sentences, `batch.text`, into the model. Note, you do not need to do `model.forward(batch.text)`, simply calling the model works. The `squeeze` is needed as the predictions are initially size _**[batch size, 1]**_, and we need to remove the dimension of size 1 as PyTorch expects the predictions input to our criterion function to be of size _**[batch size]**_.

The loss and accuracy are then calculated using our predictions and the labels, `batch.label`, with the loss being averaged over all examples in the batch.

We calculate the gradient of each parameter with `loss.backward()`, and then update the parameters using the gradients and optimizer algorithm with `optimizer.step()`.

The loss and accuracy is accumulated across the epoch, the `.item()` method is used to extract a scalar from a tensor which only contains a single value.

Finally, we return the loss and accuracy, averaged across the epoch. The `len` of an iterator is the number of batches in the iterator.

You may recall when initializing the `LABEL` field, we set `dtype=torch.float`. This is because TorchText sets tensors to be `LongTensor`s by default, however our criterion expects both inputs to be `FloatTensor`s. Setting the `dtype` to be `torch.float`, did this for us. The alternative method of doing this would be to do the conversion inside the `train` function by passing `batch.label.float()` instad of `batch.label` to the criterion.

In [55]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train() # set model - train

    for batch in iterator:

        optimizer.zero_grad() # Clear the gradient in the optimizer before each forward propagation

        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward() # Calculate the gradient of the loss by back propagation

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

`evaluate` is similar to `train`, with a few modifications as you don't want to update the parameters when evaluating.

`model.eval()` puts the model in "evaluation mode", this turns off _dropout_ and _batch normalization_. Again, we are not using them in this model, but it is good practice to include them.

No gradients are calculated on PyTorch operations inside the `with no_grad()` block. This causes less memory to be used and speeds up computation.

The rest of the function is the same as `train`, with the removal of `optimizer.zero_grad()`, `loss.backward()` and `optimizer.step()`, as we do not update the model's parameters when evaluating.

In [56]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval() # setting model - evaluate

    with torch.no_grad(): # No back propagation is required during the evaluation process, so the gradient calculation is turned off

        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

We'll also create a function to tell us how long an epoch takes to compare training times between models.

In [57]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We then train the model through multiple epochs, an epoch being a complete pass through all examples in the training and validation sets.

At each epoch, if the validation loss is the best we have seen so far, we'll save the parameters of the model and then after training has finished we'll use that model on the test set.

In [59]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.666 | Train Acc: 59.45%
	 Val. Loss: 0.673 |  Val. Acc: 57.53%
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.665 | Train Acc: 59.48%
	 Val. Loss: 0.673 |  Val. Acc: 57.48%
Epoch: 03 | Epoch Time: 0m 4s
	Train Loss: 0.665 | Train Acc: 59.46%
	 Val. Loss: 0.672 |  Val. Acc: 57.76%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss: 0.664 | Train Acc: 59.65%
	 Val. Loss: 0.671 |  Val. Acc: 57.45%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss: 0.663 | Train Acc: 59.59%
	 Val. Loss: 0.671 |  Val. Acc: 58.01%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss: 0.662 | Train Acc: 59.77%
	 Val. Loss: 0.671 |  Val. Acc: 57.95%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss: 0.662 | Train Acc: 59.79%
	 Val. Loss: 0.670 |  Val. Acc: 57.89%
Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: 0.661 | Train Acc: 59.97%
	 Val. Loss: 0.670 |  Val. Acc: 58.17%
Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: 0.660 | Train Acc: 59.92%
	 Val. Loss: 0.669 |  Val. Acc: 58.25%
Epoch: 10 | Epoch Time: 0m 4

You may have noticed the loss is not really decreasing and the accuracy is poor. This is due to several issues with the model which we'll improve in the next notebook.

Finally, the metric we actually care about, the test loss and accuracy, which we get from our parameters that gave us the best validation loss.

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

<ipython-input-28-0b8ecc81f19b>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('tut1-model.pt'))


Test Loss: 0.667 | Test Acc: 58.53%


## Next Steps

In the next notebook, the improvements we will make are:
- packed padded sequences
- pre-trained word embeddings
- different RNN architecture
- bidirectional RNN
- multi-layer RNN
- regularization
- a different optimizer

This will allow us to achieve ~84% accuracy.